In [1]:
!pip install opencv-python

In [2]:
!python -m pip install --upgrade pip

In [3]:
!pip install moviepy

In [4]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os, glob
import librosa
import librosa.display
import IPython
from IPython.display import Audio
from IPython.display import Image
import pickle
import warnings; warnings.filterwarnings('ignore') #matplot lib complains about librosa
# !pip install -U librosa
from pydub import silence, AudioSegment
import IPython.display as ipd
import  cairosvg
import  python_avatars  as  pa
import random as random
from enum import Enum
from matplotlib import image
import time
import cv2
from moviepy.editor import AudioFileClip,VideoFileClip,concatenate_videoclips
import imageio.v3 as imageio

C:\Users\future\.conda\envs\gp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
def make_test(model,criterion):
    def validate_t(X):
        with torch.no_grad():
            model.eval()
            output_logits, output_softmax = model(X)
            predictions = torch.argmax(output_softmax,dim=1)
        return predictions
    return validate_t

In [6]:
def load_checkpoint(optimizer, model, filename):
    checkpoint_dict = torch.load(filename)
    epoch = checkpoint_dict['epoch']
    model.load_state_dict(checkpoint_dict['model'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint_dict['optimizer'])
    return epoch

In [7]:
class parallel_all_you_want(nn.Module):
    # Define all layers present in the network
    def __init__(self,num_emotions):
        super().__init__()

        ################ TRANSFORMER BLOCK #############################
        # maxpool the input feature map/tensor to the transformer
        # a rectangular kernel worked better here for the rectangular input spectrogram feature map/tensor
        self.transformer_maxpool = nn.MaxPool2d(kernel_size=[1,4], stride=[1,4])

        # define single transformer encoder layer
        # self-attention + feedforward network from "Attention is All You Need" paper
        # 4 multi-head self-attention layers each with 40-->512--->40 feedforward network
        transformer_layer = nn.TransformerEncoderLayer(
            d_model=40, # input feature (frequency) dim after maxpooling 40*282 -> 40*70 (MFC*time)
            nhead=4, # 4 self-attention layers in each multi-head self-attention layer in each encoder block
            dim_feedforward=512, # 2 linear layers in each encoder block's feedforward network: dim 40-->512--->40
            dropout=0.4,
            activation='relu' # ReLU: avoid saturation/tame gradient/reduce compute time
        )

        # I'm using 4 instead of the 6 identical stacked encoder layrs used in Attention is All You Need paper
        # Complete transformer block contains 4 full transformer encoder layers (each w/ multihead self-attention+feedforward)
        self.transformer_encoder = nn.TransformerEncoder(transformer_layer, num_layers=4)

        ############### 1ST PARALLEL 2D CONVOLUTION BLOCK ############
        # 3 sequential conv2D layers: (1,40,282) --> (16, 20, 141) -> (32, 5, 35) -> (64, 1, 8)
        self.conv2Dblock1 = nn.Sequential(

            # 1st 2D convolution layer
            nn.Conv2d(
                in_channels=1, # input volume depth == input channel dim == 1
                out_channels=16, # expand output feature map volume's depth to 16
                kernel_size=3, # typical 3*3 stride 1 kernel
                stride=1,
                padding=1
                      ),
            nn.BatchNorm2d(16), # batch normalize the output feature map before activation
            nn.ReLU(), # feature map --> activation map
            nn.MaxPool2d(kernel_size=2, stride=2), #typical maxpool kernel size
            nn.Dropout(p=0.3), #randomly zero 30% of 1st layer's output feature map in training

            # 2nd 2D convolution layer identical to last except output dim, maxpool kernel
            nn.Conv2d(
                in_channels=16,
                out_channels=32, # expand output feature map volume's depth to 32
                kernel_size=3,
                stride=1,
                padding=1
                      ),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=4, stride=4), # increase maxpool kernel for subsequent filters
            nn.Dropout(p=0.3),

            # 3rd 2D convolution layer identical to last except output dim
            nn.Conv2d(
                in_channels=32,
                out_channels=64, # expand output feature map volume's depth to 64
                kernel_size=3,
                stride=1,
                padding=1
                      ),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=4, stride=4),
            nn.Dropout(p=0.3),
        )
        ############### 2ND PARALLEL 2D CONVOLUTION BLOCK ############
        # 3 sequential conv2D layers: (1,40,282) --> (16, 20, 141) -> (32, 5, 35) -> (64, 1, 8)
        self.conv2Dblock2 = nn.Sequential(

            # 1st 2D convolution layer
            nn.Conv2d(
                in_channels=1, # input volume depth == input channel dim == 1
                out_channels=16, # expand output feature map volume's depth to 16
                kernel_size=3, # typical 3*3 stride 1 kernel
                stride=1,
                padding=1
                      ),
            nn.BatchNorm2d(16), # batch normalize the output feature map before activation
            nn.ReLU(), # feature map --> activation map
            nn.MaxPool2d(kernel_size=2, stride=2), #typical maxpool kernel size
            nn.Dropout(p=0.3), #randomly zero 30% of 1st layer's output feature map in training

            # 2nd 2D convolution layer identical to last except output dim, maxpool kernel
            nn.Conv2d(
                in_channels=16,
                out_channels=32, # expand output feature map volume's depth to 32
                kernel_size=3,
                stride=1,
                padding=1
                      ),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=4, stride=4), # increase maxpool kernel for subsequent filters
            nn.Dropout(p=0.3),

            # 3rd 2D convolution layer identical to last except output dim
            nn.Conv2d(
                in_channels=32,
                out_channels=64, # expand output feature map volume's depth to 64
                kernel_size=3,
                stride=1,
                padding=1
                      ),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=4, stride=4),
            nn.Dropout(p=0.3),
            )

        ################# FINAL LINEAR BLOCK ####################
        # Linear softmax layer to take final concatenated embedding tensor
        # from parallel 2D convolutional and transformer blocks, output 8 logits
        # Each full convolution block outputs (64*1*8) embedding flattened to dim 512 1D array
        # Full transformer block outputs 40*70 feature map, which we time-avg to dim 40 1D array
        # 512*2+40 == 1064 input features --> 8 output emotions
        self.fc1_linear = nn.Linear(512*2+40,num_emotions)

        ### Softmax layer for the 8 output logits from final FC linear layer
        self.softmax_out = nn.Softmax(dim=1) # dim==1 is the freq embedding

    # define one complete parallel fwd pass of input feature tensor thru 2*conv+1*transformer blocks
    def forward(self,x):

                ############ 1st parallel Conv2D block: 4 Convolutional layers ############################
                # create final feature embedding from 1st convolutional layer
                # input features pased through 4 sequential 2D convolutional layers
                conv2d_embedding1 = self.conv2Dblock1(x) # x == N/batch * channel * freq * time

                # flatten final 64*1*8 feature map from convolutional layers to length 512 1D array
                # skip the 1st (N/batch) dimension when flattening
                conv2d_embedding1 = torch.flatten(conv2d_embedding1, start_dim=1)

                ############ 2nd parallel Conv2D block: 4 Convolutional layers #############################
                # create final feature embedding from 2nd convolutional layer
                # input features pased through 4 sequential 2D convolutional layers
                conv2d_embedding2 = self.conv2Dblock2(x) # x == N/batch * channel * freq * time

                # flatten final 64*1*8 feature map from convolutional layers to length 512 1D array
                # skip the 1st (N/batch) dimension when flattening
                conv2d_embedding2 = torch.flatten(conv2d_embedding2, start_dim=1)


                ########## 4-encoder-layer Transformer block w/ 40-->512-->40 feedfwd network ##############
                # maxpool input feature map: 1*40*282 w/ 1*4 kernel --> 1*40*70
                x_maxpool = self.transformer_maxpool(x)

                # remove channel dim: 1*40*70 --> 40*70
                x_maxpool_reduced = torch.squeeze(x_maxpool,1)

                # convert maxpooled feature map format: batch * freq * time ---> time * batch * freq format
                # because transformer encoder layer requires tensor in format: time * batch * embedding (freq)
                x = x_maxpool_reduced.permute(2,0,1)

                # finally, pass reduced input feature map x into transformer encoder layers
                transformer_output = self.transformer_encoder(x)

                # create final feature emedding from transformer layer by taking mean in the time dimension (now the 0th dim)
                # transformer outputs 2x40 (MFCC embedding*time) feature map, take mean of columns i.e. take time average
                transformer_embedding = torch.mean(transformer_output, dim=0) # dim 40x70 --> 40

                ############# concatenate freq embeddings from convolutional and transformer blocks ######
                # concatenate embedding tensors output by parallel 2*conv and 1*transformer blocks
                complete_embedding = torch.cat([conv2d_embedding1, conv2d_embedding2,transformer_embedding], dim=1)

                ######### final FC linear layer, need logits for loss #########################
                output_logits = self.fc1_linear(complete_embedding)

                ######### Final Softmax layer: use logits from FC linear, get softmax for prediction ######
                output_softmax = self.softmax_out(output_logits)

                # need output logits to compute cross entropy loss, need softmax probabilities to predict class
                return output_logits, output_softmax

In [8]:
def saved_model():
        load_folder = r"C:\Users\future\Desktop\prototype"
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        #pick the epoch to load 256 64% -
        epoch = '415'
        model_name = f'parallel_all_you_wantFINAL-{epoch}.pkl'
        load_path = os.path.join(load_folder, model_name)

        ## instantiate empty model and populate with params from binary
        model = parallel_all_you_want(num_emotions=8).to(device)
        optimizer = torch.optim.SGD(model.parameters(),lr=0.01, weight_decay=1e-3, momentum=0.8)
        print('Number of trainable params: ',sum(p.numel() for p in model.parameters()) )
        load_checkpoint(optimizer, model, load_path)
        return model

In [9]:
def load_data(test_Files):
    # raw waveforms to augment later
    waveforms = []
    for file in glob.glob(test_Files):
              waveform = get_waveforms(file)
              waveforms.append(waveform)

    return waveforms


In [10]:
def feature_mfcc(
    waveform,
    sample_rate,
    n_mfcc = 40,
    fft = 1024,
    winlen = 512,
    window='hamming',
    #hop=256, # increases # of time steps; was not helpful
    mels=128
    ):

    # Compute the MFCCs for all STFT frames
    # 40 mel filterbanks (n_mfcc) = 40 coefficients
    mfc_coefficients=librosa.feature.mfcc(
        y=waveform,
        sr=sample_rate,
        n_mfcc=n_mfcc,
        n_fft=fft,
        win_length=winlen,
        window=window,
        #hop_length=hop,
        n_mels=mels,
        fmax=sample_rate/2
        )

    return mfc_coefficients

In [11]:
def get_waveforms(file):
    sample_rate = 48000

    # load an individual sample audio file
    # read the full 3 seconds of the file, cut off the first 0.5s of silence; native sample rate = 48k
    # don't need to store the sample rate that librosa.load returns
    waveform, _ = librosa.load(file, duration=3, sr=sample_rate)

    # make sure waveform vectors are homogenous by defining explicitly
    waveform_homo = np.zeros((int(sample_rate*3,)))
    waveform_homo[:len(waveform)] = waveform

    # return a single file's waveform
    return waveform_homo

In [12]:
def get_features(waveforms, features, sample_rate):

    # initialize counter to track progress
    file_count = 0

    # process each waveform individually to get its MFCCs
    for waveform in waveforms:
        mfccs = feature_mfcc(waveform, sample_rate)
        features.append(mfccs)
        file_count += 1
        # print progress
        print('\r'+f' Processed {file_count}/{len(waveforms)} waveforms',end='')

    # return all features from list of waveforms
    return features

In [13]:
def final_test(file_test):
    
    waveforms= load_data(file_test)

    sample_rate = 48000

  # mfcc features
    features=[]
    features_t = get_features(waveforms,features, sample_rate)

    sample_test = np.expand_dims(features,1)

    N,C,H,W = sample_test.shape

    # load scaler

    path = r"C:\Users\future\Desktop\prototype\GP_Model\scaler_folder"
    os.chdir(path)
    cwd = os.getcwd()
    with open(r"C:\Users\future\Desktop\prototype\GP_Model\scaler_folder\scaler.pkl",'rb') as f:
        scaler = pickle.load(f)
    # should save scaler and then load it here to use
    sample_test= np.reshape(sample_test, (N,-1))
    sample_test= scaler.transform(sample_test)
    sample_test = np.reshape(sample_test, (N,C,H,W))

    def criterion(predictions, targets):
        return nn.CrossEntropyLoss()(input=predictions, target=targets)
    model = saved_model()
    validate_t = make_test(model,criterion)

    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    sample_test= torch.tensor(sample_test,device=device).float()

    predicted_emotion = validate_t(sample_test)
    # print(predicted_emotion)

    emotions_dict ={
      '0':'surprised',
      '1':'neutral',
      '2':'calm',
      '3':'happy',
      '4':'sad',
      '5':'angry',
      '6':'fearful',
      '7':'disgust'
    }
    
    emotions=[]
    for i in range(len(predicted_emotion)):
        e = predicted_emotion[i].item()
        # this the emotion detected from the audio
        emotion=emotions_dict[str(e)]
        emotions.append(emotion)
        print(" predicted emotion for segment ",i,": ",emotions[i])
  # # here we will delete this code because the gender will enter from the user in register
  # if (int((file_test.split("-")[6]).split(".")[0]))%2==0:
  #     gender = 'girl'
  # else:
  #     gender = 'boy'
    return emotions

In [14]:
def audio_read(audio_path):
    #cp $audio_path .
    original_audio = os.path.basename(audio_path)
    #ipd.display(ipd.Audio(original_audio))
    return original_audio

In [15]:
def read_audio(audio_path):
    audio = AudioSegment.from_file(audio_path)
    audio = audio.set_channels(1)
    return audio

def split_on_silence_with_pydub(path_audio,skip_idx=0, out_ext="wav",
        silence_thresh=-51, silence_chunk_len=300, keep_silence=300):

    audio_path=audio_read(path_audio)
    print(os.getcwd())
    filename = os.path.basename(audio_path).split('.', 1)[0]
    audio = read_audio(audio_path)

    not_silence_ranges = silence.detect_nonsilent(
        audio, min_silence_len=silence_chunk_len,
        silence_thresh=silence_thresh)
    # Save audio files
    audio_paths = []
    
    temp_path = r"C:\Users\future\Desktop\prototype\avatars_audio\segmentation"
    os.chdir(temp_path)
    cwd = os.getcwd()

    for idx, (start_idx, end_idx) in enumerate(not_silence_ranges[skip_idx:]):
        start_idx = max(0, start_idx - keep_silence)
        end_idx += keep_silence
        target_audio_path = r"C:\Users\future\Desktop\prototype\avatars_audio\segmentation\{}\{}.{:04d}.{}".format(os.path.dirname(audio_path), filename, idx, out_ext)
        segment = audio[start_idx:end_idx]
        segment.export(target_audio_path, out_ext)
        audio_paths.append(target_audio_path)


    return audio_paths,not_silence_ranges

In [16]:
def path_emotions(audio_path,gender):
        audio_list,intervals = split_on_silence_with_pydub(audio_path)
        temp_path = r"C:\Users\future\Desktop\prototype\avatars_audio\segmentation"
        os.chdir(temp_path)
        cwd = os.getcwd()
        path=r"C:\Users\future\Desktop\prototype\avatars_audio\segmentation\*.wav"
        emotions=final_test(path)
        print(emotions)
        return emotions

In [17]:
def extract_features(data, sample_rate):
    mfcc = librosa.feature.mfcc(y=data, sr=sample_rate, n_mfcc=NUM_MFCC, n_fft=N_FFT, hop_length=HOP_LENGTH)
    feature = mfcc.T
    return feature

In [18]:
from pydub import AudioSegment, effects
import noisereduce as nr
from json_tricks import load

def preprocess(file_path, SAMPLE_RATE = 48000, frame_length = 2048, N_FFT = 2048, hop_length = 512):

     # Fetch sample rate.
    _, sr = librosa.load(path = file_path, sr = None)
    # Load audio file
    rawsound = AudioSegment.from_file(file_path, duration = None)
    # Normalize to 5 dBFS
    normalizedsound = effects.normalize(rawsound, headroom = 5.0)
    # Transform the audio file to np.array of samples
    normal_x = np.array(normalizedsound.get_array_of_samples(), dtype = 'float32')
    # Noise reduction
    final_x = nr.reduce_noise(normal_x, sr=sr)
    # extract features
    f3 = extract_features(final_x,sr) # MFCC

    return f3

In [19]:
import pyaudio
import wave
from array import array
import struct
import time
import os
import glob

# to record audio using realtime

def recordAudio():
        RATE = 48000
        CHUNK = 40960
        RECORD_SECONDS = 10

        FORMAT = pyaudio.paInt32
        CHANNELS = 1
        WAVE_OUTPUT_FILE = r"C:\Users\future\Desktop\prototype\outputtt.wav"

        # Open an input channel
        p = pyaudio.PyAudio()
        stream = p.open(format=FORMAT,
                  channels=CHANNELS,
                  rate=RATE,
                  input=True,
                  frames_per_buffer=CHUNK)

        chunk = 512  # Record in chunks of 1024 samples
        chunk_counter = 0
        sample_format = pyaudio.paInt16  # 16 bits per sample
        channels = 1
        fs = 48000 #16000  # Record at 44100 samples per second //as per ravdess dataset the frequecy is 48kHz
        seconds = 10
        filename = "audio_chunk"
        audio_path = r"C:/Users/future/Desktop/prototype/Realtime/*.wav"

        p = pyaudio.PyAudio()  # Create an interface to PortAudio

        print('Recording')

        stream = p.open(format=sample_format,
                    channels=channels,
                    rate=fs,
                    frames_per_buffer=chunk,
                    input=True)

        frames = []  # Initialize array to store frames

        # Store data in chunks for 10 seconds
        for i in range(0, int(fs / chunk * seconds)):
            data = stream.read(chunk)
            frames.append(data)

        # Stop and close the stream
        stream.stop_stream()
        stream.close()
        # Terminate the PortAudio interface
        p.terminate()

        print('Finished recording')

       # Save the recorded data as a WAV file
        wf = wave.open(f"{filename}_{chunk_counter}.wav", 'wb')
        wf.setnchannels(channels)
        wf.setsampwidth(p.get_sample_size(sample_format))
        wf.setframerate(fs)
        wf.writeframes(b''.join(frames))
        wf.close()

In [20]:
audio_path=r"C:\Users\future\Desktop\prototype\Realtime"
os.chdir(audio_path)
cwd = os.getcwd()

In [21]:
def avatar(EB,E,M,TYPE,personality):
    if personality == "boy":
        C = pa.ClothingType.BLAZER_SHIRT
        T = pa.HairType.SHORT_ROUND
    else:
        C= pa.ClothingType.GLADIATOR
        T= pa.HairType.LOOSE_HAIR

    my_avatar = pa.Avatar(
          style=pa.AvatarStyle.CIRCLE,
          background_color=pa.BackgroundColor.BLACK,
          top=T,
          eyebrows=EB,
          eyes=E,
          nose=pa.NoseType.DEFAULT,
          mouth=M,
          facial_hair=pa.FacialHairType.NONE,
          skin_color="#FFDAB9",
          hair_color=pa.HairColor.BLACK,
          accessory=pa.AccessoryType.NONE,
          clothing=C,
          clothing_color=pa.ClothingColor
      )

    temp_path = r"C:\Users\future\Desktop\prototype\avatars"
    os.chdir(temp_path)
    cwd = os.getcwd()
    # Save to a file
    my_avatar.render(r"C:\Users\future\Desktop\prototype\avatars\\"+TYPE+"_"+personality+".svg")
    example_url =r"C:\Users\future\Desktop\prototype\avatars\\"+TYPE+'_'+personality+'.svg'
    path=r"C:\Users\future\Desktop\prototype\avatars\\"+TYPE+'_'+personality+'.png'
    s = cairosvg.svg2png(url=example_url,write_to=path)

In [22]:
## disgust
def disgust(type):
    avatar(pa.EyebrowType.DEFAULT_NATURAL,pa.EyeType.SIDE,pa.MouthType.CONCERNED,"disgust",type)
    avatar(pa.EyebrowType.DEFAULT_NATURAL,pa.EyeType.SIDE,pa.MouthType.SERIOUS,"natural_d",type)

In [23]:
## surprise
def surprised(type):
    avatar(pa.EyebrowType.DEFAULT_NATURAL,pa.EyeType.SURPRISED,pa.MouthType.DISBELIEF,"surprised",type)
    avatar(pa.EyebrowType.DEFAULT_NATURAL,pa.EyeType.SURPRISED,pa.MouthType.SERIOUS,"natural_sur",type)

In [24]:
## fearful
def fearful(type):
    avatar(pa.EyebrowType.DEFAULT_NATURAL,pa.EyeType.SQUINT,pa.MouthType.SCREAM_OPEN,"fearful",type)
    avatar(pa.EyebrowType.DEFAULT_NATURAL,pa.EyeType.SQUINT,pa.MouthType.SERIOUS,"natural_f",type)

In [25]:
## angry
def angry(type):
    avatar(pa.EyebrowType.ANGRY,pa.EyeType.SQUINT,pa.MouthType.GRIMACE,"angry",type)
    avatar(pa.EyebrowType.ANGRY,pa.EyeType.SQUINT,pa.MouthType.SERIOUS,"natural_a",type)

In [26]:
## calm
def calm(type):
    avatar(pa.EyebrowType.DEFAULT_NATURAL,pa.EyeType.CLOSED,pa.MouthType.TWINKLE,"calm",type)
    avatar(pa.EyebrowType.DEFAULT_NATURAL,pa.EyeType.CLOSED,pa.MouthType.SERIOUS,"natural_c",type)

In [27]:
## happy
def happy(type):
    avatar(pa.EyebrowType.DEFAULT_NATURAL,pa.EyeType.HAPPY,pa.MouthType.SMILE,"happy",type)
    avatar(pa.EyebrowType.DEFAULT_NATURAL,pa.EyeType.HAPPY,pa.MouthType.SERIOUS,"natural_h",type)

In [28]:
## sad
def sad(type):
    avatar(pa.EyebrowType.SAD_CONCERNED_NATURAL,pa.EyeType.CRY,pa.MouthType.SAD,"sad",type)
    avatar(pa.EyebrowType.SAD_CONCERNED_NATURAL,pa.EyeType.CRY,pa.MouthType.SERIOUS,"natural_s",type)

In [29]:
###UPDATED
## neutral
def neutral(type):
    avatar(pa.EyebrowType.DEFAULT_NATURAL,pa.EYeType.DEFAULT,pa.MouthType.TWINKLE,"neutral",type)
    avatar(pa.EyebrowType.DEFAULT_NATURAL,pa.EYeType.DEFAULT,MouthType.SERIOUS,"natural_n",type)

In [30]:
### Updated
def choice(emotion,type):
        N=" "
        if emotion=="disgust":
            disgust(type)
            N="d"
        elif emotion=="surprised":
            surprised(type)
            N="sur"
        elif emotion=="fearful":
            fearful(type)
            N="f"
        elif emotion=="angry":
            angry(type)
            N="a"
        elif emotion=="calm":
            calm(type)
            N="c"
        elif emotion=="happy":
            happy(type)
            N="h"
        elif emotion=="sad":
            sad(type)
            N="s"
        elif emotion=="neutral":
            neutral(type)
            N="n"
        return N

In [31]:
#type="boy"
#emotion="happy"
def show(emotion,type):
    N=choice(emotion,type)
    path = r"C:\Users\future\Desktop\prototype\avatars"
    os.chdir(path)
    cwd = os.getcwd()
    path1=r"C:\Users\future\Desktop\prototype\avatars\\"+emotion+"_"+type+".png"
    # im=image.imread(path1)
    # plt.imshow(im)
    # plt.axis("off")
    # plt.show(block=False)
    # plt.close()
    # time.sleep(1)
    path = r"C:\Users\future\Desktop\prototype\avatars"
    os.chdir(path)
    cwd = os.getcwd()
    path2=r"C:\Users\future\Desktop\prototype\avatars\natural_"+N+'_'+type+".png"
    # im=image.imread(path2)
    # plt.imshow(im)
    # plt.axis("off")
    # plt.show()
    return path1,path2

In [32]:
def show_avatar(file_test,emotion,gender,saved_file):
    path1,path2=show(emotion,gender)
    images = []
    # here will take the input audio
    audio_clip = AudioFileClip(file_test)
    a=audio_clip.duration
    print(a)
    # here take paths of the emojies
    for filename in range(int(a*2)):
        images.append(imageio.imread(path1))
        images.append(imageio.imread(path2))
    # Create the output video file
    # here will take the path to save on it the video
    imageio.imwrite(saved_file, images, fps=5)
    video_clip = VideoFileClip(saved_file)
    final_clip = video_clip.set_audio(audio_clip)
    # Export the final video with audio
    final_clip.write_videofile(saved_file)

In [33]:
def recorded_final_video(gender,audio_path):
    emotions=path_emotions(audio_path,gender)
    folder=r"C:\Users\future\Desktop\prototype\avatars_audio\segmentation"
    os.chdir(folder)
    cwd = os.getcwd()
    all_saved_files=[]
    for i in range(len(emotions)):
        num_audio='{:04d}'.format(i)
        file_name=os.path.splitext(os.path.basename(audio_path))[0]
        model_name = file_name+f'.{num_audio}.wav'
        temp_path = r"C:\Users\future\Desktop\prototype\avatars_audio\small_audios"
        os.chdir(temp_path)
        cwd = os.getcwd()
        saved_file=r"C:\Users\future\Desktop\prototype\avatars_audio\small_audios\output{:04d}.mp4".format(i)
        all_saved_files.append(saved_file)
        load_path = os.path.join(folder, model_name)
        show_avatar(load_path,emotions[i],gender,saved_file)
    video_clips=[VideoFileClip(path)for path in all_saved_files]
    conactenated_clips=concatenate_videoclips(video_clips,method="compose")
    path = r"C:\Users\future\Desktop\prototype\avatars_audio"
    os.chdir(path)
    cwd = os.getcwd()
    output_path=r"C:\Users\future\Desktop\prototype\avatars_audio\last_video.mp4"
    conactenated_clips.write_videofile(output_path)


In [34]:
def realtime_final_video(gender,audio_path):
    emotions=path_emotions(audio_path,gender)
    folder=r"C:\Users\future\Desktop\prototype\avatars_audio\segmentation"
    os.chdir(folder)
    cwd = os.getcwd()
    all_saved_files=[]
    for i in range(len(emotions)):
        num_audio='{:04d}'.format(i)
        file_name=os.path.splitext(os.path.basename(audio_path))[0]
        model_name = file_name+f'.{num_audio}.wav'
        temp_path = r"C:\Users\future\Desktop\prototype\avatars_audio\small_audios"
        os.chdir(temp_path)
        cwd = os.getcwd()
        saved_file=r"C:\Users\future\Desktop\prototype\avatars_audio\small_audios\output{:04d}.mp4".format(i)
        all_saved_files.append(saved_file)
        load_path = os.path.join(folder, model_name)
        show_avatar(load_path,emotions[i],gender,saved_file)
    video_clips=[VideoFileClip(path)for path in all_saved_files]
    conactenated_clips=concatenate_videoclips(video_clips,method="compose")
    #path = r"C:\Users\future\Desktop\prototype\Realtime"
    #os.chdir(path)
    output_path=r"C:\Users\future\Desktop\prototype\videos\last_video.mp4"
    conactenated_clips.write_videofile(output_path)


In [35]:
def realTime(audio_path,gender):
    try:
        while True:
            recordAudio()
            # wait until audio is saved
            time.sleep(2.5)
            # generating video for the recorded audio
            realtime_final_video(gender,audio_path)
            # delete the old audio before saving the new one
            files = glob.glob('/Realtime/*')
            for f in files:
                os.remove(f)
    # Cation: interupt it manaually as the except is not working and your cpu will be burnt out (:
    except KeyboardInterrupt:
            print("Recording stopped.")
            # generating video for the last recorded audio
            realtime_final_video(gender,audio_path)
            # delete the last generated video
            files = glob.glob('/videos/*')
            for f in files:
                os.remove(f)

In [36]:
audio_path=r"C:\Users\future\Desktop\prototype\avatars_audio"
os.chdir(audio_path)
cwd = os.getcwd()

In [37]:
!pip install moviepy --upgrade

In [38]:
!pip install ffmpeg --upgrade

In [39]:
!pip install decorator --upgrade --force-reinstall

  Using cached decorator-5.1.1-py3-none-any.whl.metadata (4.0 kB)
Using cached decorator-5.1.1-py3-none-any.whl (9.1 kB)
  Attempting uninstall: decorator
    Found existing installation: decorator 4.4.2
    Uninstalling decorator-4.4.2:
      Successfully uninstalled decorator-4.4.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
moviepy 1.0.3 requires decorator<5.0,>=4.0.2, but you have decorator 5.1.1 which is incompatible.


In [40]:
!pip install --upgrade decorator==4.4.2

  Using cached decorator-4.4.2-py2.py3-none-any.whl.metadata (4.2 kB)
Using cached decorator-4.4.2-py2.py3-none-any.whl (9.2 kB)
  Attempting uninstall: decorator
    Found existing installation: decorator 5.1.1
    Uninstalling decorator-5.1.1:
      Successfully uninstalled decorator-5.1.1


In [41]:
!pip install ffmpeg --upgrade

In [42]:
audio_path=r"C:\Users\future\Desktop\prototype\Realtime"
os.chdir(audio_path)
cwd = os.getcwd()

In [43]:
gender='girl'
audio_path=r"C:\Users\future\Desktop\prototype\Realtime\audio_chunk_0.wav"
realTime(audio_path,gender)
realtime_final_video(gender,audio_path)

Recording
Finished recording
C:\Users\future\Desktop\prototype\Realtime
 Processed 10/10 waveformsNumber of trainable params:  248488
 predicted emotion for segment  0 :  calm
 predicted emotion for segment  1 :  calm
 predicted emotion for segment  2 :  disgust
 predicted emotion for segment  3 :  calm
 predicted emotion for segment  4 :  calm
 predicted emotion for segment  5 :  calm
 predicted emotion for segment  6 :  calm
 predicted emotion for segment  7 :  sad
 predicted emotion for segment  8 :  calm
 predicted emotion for segment  9 :  calm
['calm', 'calm', 'disgust', 'calm', 'calm', 'calm', 'calm', 'sad', 'calm', 'calm']
8.44


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (264, 280) to (272, 288) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Moviepy - Building video C:\Users\future\Desktop\prototype\avatars_audio\small_audios\output0000.mp4.
MoviePy - Writing audio in output0000TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video C:\Users\future\Desktop\prototype\avatars_audio\small_audios\output0000.mp4



Moviepy - Done !
Moviepy - video ready C:\Users\future\Desktop\prototype\avatars_audio\small_audios\output0000.mp4


...

0.6
Moviepy - Building video C:\Users\future\Desktop\prototype\avatars_audio\small_audios\output0009.mp4.
MoviePy - Writing audio in output0009TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video C:\Users\future\Desktop\prototype\avatars_audio\small_audios\output0009.mp4



Moviepy - Done !
Moviepy - video ready C:\Users\future\Desktop\prototype\avatars_audio\small_audios\output0009.mp4
Moviepy - Building video C:\Users\future\Desktop\prototype\videos\last_video.mp4.
MoviePy - Writing audio in last_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video C:\Users\future\Desktop\prototype\videos\last_video.mp4



Moviepy - Done !
Moviepy - video ready C:\Users\future\Desktop\prototype\videos\last_video.mp4
